## Plotting Time Series Data from NetCDF

In this example we show how to programatically download and work with OOI NetCDF time series data. We will use data from the Global Argentine Basin Flanking Subsurface Mooring A Mooring Riser CTD at 30 m. You will learn:

* how to find the data you are looking for
* how to use the machine to machine API to request data
* how to load the NetCDF data into your notebook, once the data request has completed
* how to plot data

For the instrument in this example, you will need the Reference Designator, Stream and Data Delivery Method to make the request to the M2M API. More information about the instrument can be found here:
http://ooi.visualocean.net/instruments/view/GA03FLMA-RIM01-02-CTDMOG040

![GA03FLMA-RIM01-02-CTDMOG040](https://github.com/ooi-data-review/ooi_datateam_notebooks/raw/master/images/GA03FLMA-RIM01-02-CTDMOG040.png)

In [ ]:
import requests
import time

Login in at https://ooinet.oceanobservatories.org/ and obtain your <b>API username and API token</b> under your profile (top right corner), or use the credentials provided below.

In [ ]:
username = 'OOIAPI-D8S960UXPK4K03'
token = 'IXL48EQ2XY'

Specify your inputs.

In [ ]:
subsite = 'GA03FLMA'
node = 'RIM01'
sensor = '02-CTDMOG040'
method = 'recovered_inst'
stream = 'ctdmo_ghqr_instrument_recovered'
beginDT = '2015-03-17T01:01:01.500Z' # this time range encompasses all three deployments of this instrument.
endDT = '2018-09-01T01:01:01.500Z'

Build the GET request URL and send the request to the M2M API endpoint.

In [ ]:
base_url = 'https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/'

data_request_url ='/'.join((base_url,subsite,node,sensor,method,stream))
params = {
    'beginDT':beginDT,
    'endDT':endDT,   
}
r = requests.get(data_request_url, params=params, auth=(username, token))
data = r.json()

In [ ]:
data['allURLs'][0]

Check if the request has completed.

In [ ]:
%%time
check_complete = data['allURLs'][1] + '/status.txt'
for i in range(1800): 
    r = requests.get(check_complete)
    if r.status_code == requests.codes.ok:
        print('request completed')
        break
    else:
        time.sleep(1)

In [ ]:
import re
import xarray as xr
import pandas as pd
import os

Parse the thredds server to get a list of all NetCDF files. Each deployment is seperated into a seperate netcdf file.

In [ ]:
url = data['allURLs'][0]
tds_url = 'https://opendap.oceanobservatories.org/thredds/dodsC'
datasets = requests.get(url).text
urls = re.findall(r'href=[\'"]?([^\'" >]+)', datasets)
x = re.findall(r'(ooi/.*?.nc)', datasets)
for i in x:
    if i.endswith('.nc') == False:
        x.remove(i)
for i in x:
    try:
        float(i[-4])
    except:
        x.remove(i)
datasets = [os.path.join(tds_url, i) for i in x]
datasets

Load all files into a single xarray dataset.

In [ ]:
ds = xr.open_mfdataset(datasets)
ds = ds.swap_dims({'obs': 'time'})
ds = ds.chunk({'time': 100})
ds = ds.sortby('time') # data from different deployments can overlap so we want to sort all data by time stamp.
ds

Check sampling frequency.

In [ ]:
df = ds.to_dataframe()
res = (pd.Series(df.index[1:]) - pd.Series(df.index[:-1])).value_counts()
res

Plot the dataset using matplotlib, working under the hood of xarray.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(1)
fig.set_size_inches(16, 6)

ds['ctdmo_seawater_pressure'].plot(linestyle = 'None', 
                                   marker='.', 
                                   markersize=1,
                                   ax=ax)

ax.invert_yaxis()
ax.grid()

plt.title('Global Argentine Basin Flanking Subsurface Mooring A Mooring Riser CTD (30 m)')
plt.ylabel('Pressure (dbar)')
plt.xlabel('Time \n Deployments 1 - 3')
plt.show()

This CTD is supposed to be stationary at around 30 m depth in the water column, but violent storms in the southern himisphere can draw the CTD mooring chain down by hundreds of meters. More information about the Flanking Subsurface Mooring A can be found [here](http://oceanobservatories.org/site/ga03flma/).

In [ ]:
# Import Bokeh functions
import os
from bokeh.plotting import figure, output_file, reset_output, show, ColumnDataSource, save
from bokeh.layouts import column
from bokeh.models import BoxAnnotation
from bokeh.io import output_notebook

In [ ]:
source = ColumnDataSource(
    data=dict(
        x=list(ds.time.values),
        y=list((ds.ctdmo_seawater_pressure.values*-1))
    )
)

s = figure(width=950,
           height=500,
           title='Global Argentine Basin Flanking Subsurface Mooring A Mooring Riser CTD (30 m)',
           x_axis_label='Time (GMT)',
           y_axis_label='Pressure (dbar)',
           x_axis_type='datetime')

s.line('x', 'y', line_width=3, source=source)
s.circle('x', 'y', fill_color='white', size=4, source=source)

output_notebook()
show(column(s))

# output_file(os.getcwd())
# save(s, filename='pressure.html')